In [1]:
!pip install deepchem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 33.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import deepchem as dc
from rdkit import Chem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression




import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


/kaggle/input/bioactivity-data/PPARg_bioactivity_data_nM.csv
/kaggle/input/bioactivity-data/PPARg_bioactivity_data_no_missing_data_values .csv


In [3]:
biodata = pd.read_csv('/kaggle/input/bioactivity-data/PPARg_bioactivity_data_nM.csv')
biomiss = pd.read_csv('/kaggle/input/bioactivity-data/PPARg_bioactivity_data_no_missing_data_values .csv')

# se på data

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
biodata.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,NaN,32233,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL111217,NaN,CHEMBL111217,7.20,0,http://www.openphacts.org/units/Nanomolar,251326,=,1,1,=,NaN,EC50,nM,NaN,63.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,63.0
1,NaN,32234,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL111217,NaN,CHEMBL111217,8.19,0,http://www.openphacts.org/units/Nanomolar,251326,=,1,1,=,NaN,EC50,nM,NaN,6.4,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,6.4
2,NaN,33386,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(O...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL130231,NaN,CHEMBL130231,7.29,0,http://www.openphacts.org/units/Nanomolar,251310,=,1,1,=,NaN,EC50,nM,NaN,51.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,51.0
3,NaN,33387,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(O...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL130231,NaN,CHEMBL130231,8.57,0,http://www.openphacts.org/units/Nanomolar,251310,=,1,1,=,NaN,EC50,nM,NaN,2.7,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,2.7
4,NaN,35892,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,C/C(=C/C=C/C(C)=C/C(=O)O)c1cc(-c2cc(F)cc(F)c2)...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL337393,NaN,CHEMBL337393,7.40,0,http://www.openphacts.org/units/Nanomolar,251312,=,1,1,=,NaN,EC50,nM,NaN,40.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,40.0


In [6]:
biodata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   activity_comment           50 non-null     float64
 1   activity_id                1445 non-null   int64  
 2   activity_properties        1445 non-null   object 
 3   assay_chembl_id            1445 non-null   object 
 4   assay_description          1445 non-null   object 
 5   assay_type                 1445 non-null   object 
 6   assay_variant_accession    0 non-null      float64
 7   assay_variant_mutation     0 non-null      float64
 8   bao_endpoint               1445 non-null   object 
 9   bao_format                 1445 non-null   object 
 10  bao_label                  1445 non-null   object 
 11  canonical_smiles           1445 non-null   object 
 12  data_validity_comment      4 non-null      object 
 13  data_validity_description  4 non-null      objec

In [7]:
biomiss.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,NaN,32233,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL111217,NaN,CHEMBL111217,7.20,0,http://www.openphacts.org/units/Nanomolar,251326,=,1,1,=,NaN,EC50,nM,NaN,63.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,0.0630
1,NaN,32234,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL111217,NaN,CHEMBL111217,8.19,0,http://www.openphacts.org/units/Nanomolar,251326,=,1,1,=,NaN,EC50,nM,NaN,6.4,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,0.0064
2,NaN,33386,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(O...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL130231,NaN,CHEMBL130231,7.29,0,http://www.openphacts.org/units/Nanomolar,251310,=,1,1,=,NaN,EC50,nM,NaN,51.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,0.0510
3,NaN,33387,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(O...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL130231,NaN,CHEMBL130231,8.57,0,http://www.openphacts.org/units/Nanomolar,251310,=,1,1,=,NaN,EC50,nM,NaN,2.7,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,0.0027
4,NaN,35892,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,C/C(=C/C=C/C(C)=C/C(=O)O)c1cc(-c2cc(F)cc(F)c2)...,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL337393,NaN,CHEMBL337393,7.40,0,http://www.openphacts.org/units/Nanomolar,251312,=,1,1,=,NaN,EC50,nM,NaN,40.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,0.0400


In [8]:
biomiss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   activity_comment           50 non-null     float64
 1   activity_id                1445 non-null   int64  
 2   activity_properties        1445 non-null   object 
 3   assay_chembl_id            1445 non-null   object 
 4   assay_description          1445 non-null   object 
 5   assay_type                 1445 non-null   object 
 6   assay_variant_accession    0 non-null      float64
 7   assay_variant_mutation     0 non-null      float64
 8   bao_endpoint               1445 non-null   object 
 9   bao_format                 1445 non-null   object 
 10  bao_label                  1445 non-null   object 
 11  canonical_smiles           1445 non-null   object 
 12  data_validity_comment      4 non-null      object 
 13  data_validity_description  4 non-null      objec

In [9]:
len(biodata)

1445

In [10]:
len(biomiss)

1445

In [11]:
biodata.isna().sum()

activity_comment             1395
activity_id                     0
activity_properties             0
assay_chembl_id                 0
assay_description               0
assay_type                      0
assay_variant_accession      1445
assay_variant_mutation       1445
bao_endpoint                    0
bao_format                      0
bao_label                       0
canonical_smiles                0
data_validity_comment        1441
data_validity_description    1441
document_chembl_id              0
document_journal               50
document_year                   0
ligand_efficiency             748
molecule_chembl_id              0
molecule_pref_name           1336
parent_molecule_chembl_id       0
pchembl_value                 172
potential_duplicate             0
qudt_units                     61
record_id                       0
relation                        0
src_id                          0
standard_flag                   0
standard_relation               0
standard_text_

In [12]:
biomiss.isna().sum()

activity_comment             1395
activity_id                     0
activity_properties             0
assay_chembl_id                 0
assay_description               0
assay_type                      0
assay_variant_accession      1445
assay_variant_mutation       1445
bao_endpoint                    0
bao_format                      0
bao_label                       0
canonical_smiles                0
data_validity_comment        1441
data_validity_description    1441
document_chembl_id              0
document_journal               50
document_year                   0
ligand_efficiency             748
molecule_chembl_id              0
molecule_pref_name           1336
parent_molecule_chembl_id       0
pchembl_value                 172
potential_duplicate             0
qudt_units                     61
record_id                       0
relation                        0
src_id                          0
standard_flag                   0
standard_relation               0
standard_text_

**Biodata og biomiss er ser helt like ut, så velger bare biodata**

# prepare data


**dropper dårlig data**

In [13]:
#Jeg dropper featurså jeg tror er urelevant. 
biodata.drop(["activity_comment", "data_validity_comment", "data_validity_description", "document_journal", "document_year", "text_value", "activity_id", "assay_chembl_id", "bao_label", "document_chembl_id", "molecule_chembl_id",
             "parent_molecule_chembl_id", "record_id", "standard_value"],axis=1, inplace=True)


In [14]:
#dropper de med for mange manglende verdier.
biodata.drop(["assay_variant_accession", "assay_variant_mutation", "molecule_pref_name","standard_text_value", "standard_upper_value", "toid", "upper_value", "ligand_efficiency",],axis=1, inplace=True)


In [15]:
#dropper de kolonnene som har manglende verdier.
biodata = biodata.dropna(subset=["pchembl_value", "qudt_units", "uo_units"], axis=0)

In [16]:
#sjekker om radene har de samme verdiene, fordi da er det ikke vits i å ha de med
unik = biodata.nunique()
unikkol = unik[unik == 1].index
print(unikkol.tolist())

['activity_properties', 'bao_endpoint', 'qudt_units', 'relation', 'standard_flag', 'standard_relation', 'standard_type', 'standard_units', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units']


In [17]:
#dropper de har alle radene som er like 
biodata.drop(['activity_properties', 'bao_endpoint', 'qudt_units', 'standard_relation', 'standard_type', 'standard_units', 'relation', 'standard_flag', 
              'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units'], axis=1, inplace=True)

**få numerisk data**

In [18]:
#assay_type og bao_format
oe = OneHotEncoder()
assay_type_bao_format_encoded = oe.fit_transform(biodata[['assay_type', 'bao_format']]).toarray()
oekol = oe.get_feature_names_out(['assay_type', 'bao_format'])

In [19]:
le = LabelEncoder()
src_id_encoded = le.fit_transform(biodata['src_id'])

In [20]:
#molekylsk struktur til numerisk
def smiles_to_morgan(smiles, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    fingerprint = dc.feat.CircularFingerprint(radius=2, size=n_bits).featurize([mol])[0]
    return fingerprint

morgan_fps = biodata['canonical_smiles'].apply(smiles_to_morgan)

In [21]:
#slå ilag prepared data
preprocessed_data = pd.concat([pd.DataFrame(assay_type_bao_format_encoded, columns=oekol),
                                biodata[['pchembl_value', 'potential_duplicate', 'value']],
                                pd.DataFrame(src_id_encoded, columns=['src_id']),
                                pd.DataFrame(morgan_fps.tolist())], axis=1)

In [22]:
#dropper NaN verdier
preprocessed_data = preprocessed_data.dropna()

# modell

In [23]:
X = preprocessed_data.drop(columns=['pchembl_value'])
y = preprocessed_data['pchembl_value']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
#prøver å priorotere de "viktigste featurså"
selector = SelectKBest(score_func=f_regression, k=30)  # Choose the best 30 features
Z = selector.fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(Z, y, test_size=0.2, random_state=42)


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in true_divide
  correlation_coefficient /= X_norms
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in true_divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got featur

In [26]:
#antar rfr er best
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [27]:
#modellen overtilpasser så prøver kryssvalidering
#r2_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
#print(f"Mean R2-score: {np.mean(r2_scores):.2f}")
#print(f"R2-score standard deviation: {np.std(r2_scores):.2f}")
#....men det ser ikke ut til å gjøre det bedre.

In [28]:
#prøver å tune hyperparameter
#param_grid = {
#    'n_estimators': [50, 100, 150, 200],
#    'max_depth': [None, 10, 20, 30],
#    'min_samples_split': [2, 5, 10],
#    'min_samples_leaf': [1, 2, 4]
#}

#model = RandomForestRegressor(random_state=42)
#grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
#grid_search.fit(X, y)

#print(f"Best hyperparameters: {grid_search.best_params_}")
#print(f"Best R2-score: {grid_search.best_score_:.2f}")
#ser ikke ut til å bli bedre....

In [29]:
y_pred = model.predict(X_test)

In [30]:
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R2-score: {r2:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")

R2-score: 1.00
Mean Squared Error: 0.00
Mean Absolute Error: 0.01
